# COVID-19 India Data Deep Dive

In [52]:
import numpy as np
import pandas as pd
import sqlite3 as sql

In [53]:
data = pd.read_csv('/Users/vaishnavi/Downloads/Latest Covid-19 India Status.csv')

In [54]:
#first 5 rows of data
data.head()

,State/UTs,Total Cases,Active,Discharged,Deaths,Active Ratio,Discharge Ratio,Death Ratio,Population
0,Andaman and Nicobar,10747,0,10618,129,0.0,98.80,1.20,100896618
1,Andhra Pradesh,2339078,7,2324338,14733,0.0,99.37,0.63,128500364
2,Arunachal Pradesh,66891,0,66595,296,0.0,99.56,0.44,658019
3,Assam,746100,0,738065,8035,0.0,98.92,1.08,290492
4,Bihar,851404,1,839100,12303,0.0,98.55,1.45,40100376


In [55]:
#rename the filename and columns so the dataset is SQL compatible
data_copy = data
data_copy.to_csv('IndiaCovidData.csv', index = False)

data_sql = pd.read_csv('IndiaCovidData.csv')

data_sql = data_sql.rename(columns = 
                          {'State/UTs': 'States',
                           'Total Cases': 'TotalCases',
                          'Active Ratio': 'ActiveRatio',
                          'Discharge Ratio': 'DischargeRatio',
                          'Death Ratio': 'DeathRatio'}, inplace = False)
data_sql.head()

,States,TotalCases,Active,Discharged,Deaths,ActiveRatio,DischargeRatio,DeathRatio,Population
0,Andaman and Nicobar,10747,0,10618,129,0.0,98.80,1.20,100896618
1,Andhra Pradesh,2339078,7,2324338,14733,0.0,99.37,0.63,128500364
2,Arunachal Pradesh,66891,0,66595,296,0.0,99.56,0.44,658019
3,Assam,746100,0,738065,8035,0.0,98.92,1.08,290492
4,Bihar,851404,1,839100,12303,0.0,98.55,1.45,40100376


In [56]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

data_sql.to_sql("IndiaCovidData", con =engine)

data_sql.to_sql('IndiaCovidData', con=engine, if_exists='append', index=False)
data_sql.head()

,States,TotalCases,Active,Discharged,Deaths,ActiveRatio,DischargeRatio,DeathRatio,Population
0,Andaman and Nicobar,10747,0,10618,129,0.0,98.80,1.20,100896618
1,Andhra Pradesh,2339078,7,2324338,14733,0.0,99.37,0.63,128500364
2,Arunachal Pradesh,66891,0,66595,296,0.0,99.56,0.44,658019
3,Assam,746100,0,738065,8035,0.0,98.92,1.08,290492
4,Bihar,851404,1,839100,12303,0.0,98.55,1.45,40100376


In [57]:
# Query 1 - first 5 rows of the dataset
sql = '''

SELECT *

FROM IndiaCovidData

'''

data_sql = pd.read_sql_query(sql,con=engine)
data_sql.head()

,index,States,TotalCases,Active,Discharged,Deaths,ActiveRatio,DischargeRatio,DeathRatio,Population
0,0.0,Andaman and Nicobar,10747,0,10618,129,0.0,98.80,1.20,100896618
1,1.0,Andhra Pradesh,2339078,7,2324338,14733,0.0,99.37,0.63,128500364
2,2.0,Arunachal Pradesh,66891,0,66595,296,0.0,99.56,0.44,658019
3,3.0,Assam,746100,0,738065,8035,0.0,98.92,1.08,290492
4,4.0,Bihar,851404,1,839100,12303,0.0,98.55,1.45,40100376


In [58]:
# Query 2 - least affected states to most affected states
sql = '''

SELECT States, Active

FROM IndiaCovidData

GROUP BY Active

ORDER BY Active asc

'''

data_sql = pd.read_sql_query(sql,con=engine)
data_sql

,States,Active
0,Andaman and Nicobar,0
1,Bihar,1
2,Madhya Pradesh,2
3,Chandigarh,3
4,Rajasthan,5
5,Andhra Pradesh,7
6,Chhattisgarh,8
7,Delhi,10
8,Gujarat,11
9,Himachal Pradesh,14


In [59]:
# Query 3 - the state with the most and least cases 
sql = '''

SELECT * FROM

(SELECT 'Min : ' || States, min(Active) as MaxMinCases FROM IndiaCovidData
union all 
SELECT 'Max : ' || States, max(Active) as MaxMinCases FROM IndiaCovidData)

'''
data_sql = pd.read_sql_query(sql,con=engine)
data_sql

,'Min : ' || States,MaxMinCases
0,Min : Andaman and Nicobar,0
1,Max : Kerala,1300


In [60]:
# Query 4 - the state with the greatest rate of cases and least rate of cases
sql = '''

SELECT * FROM (
SELECT States, min(RateOfActiveCases) as MaxMinRates FROM
(SELECT  States, CAST(Active AS FLOAT)/CAST(Population AS FLOAT) as RateOfActiveCases FROM IndiaCovidData)
)
union all
SELECT * FROM (
SELECT States, max(RateOfActiveCases) as MaxMinRates FROM
(SELECT  States, CAST(Active AS FLOAT)/CAST(Population AS FLOAT) as RateOfActiveCases FROM IndiaCovidData)
)

'''
data_sql = pd.read_sql_query(sql,con=engine)
data_sql

,States,MaxMinRates
0,Andaman and Nicobar,0.000000
1,Maharashtra,0.000336


In [61]:
# Query 5 - ranking by population, rate of cases to assess if states with greater population have higher rates contraction
sql = '''

SELECT States, Population, Active

FROM IndiaCovidData

ORDER BY Active desc


'''
data_sql = pd.read_sql_query(sql,con=engine)
data_sql

,States,Population,Active
0,Kerala,91702478,1300
1,Kerala,91702478,1300
2,Maharashtra,399001,134
3,Maharashtra,399001,134
4,Karnataka,1711947,123
...,...,...,...
67,Lakshadweep,11700099,0
68,Manipur,47099270,0
69,Mizoram,1308967,0
70,Nagaland,38157311,0


In [62]:
# Query 6 - the rate of cases not discharged
sql = '''

SELECT * FROM (
SELECT States, notDischarged FROM
(SELECT  States, 1 - (CAST(Discharged AS FLOAT)/CAST(TotalCases AS FLOAT)) as notDischarged 
FROM IndiaCovidData
ORDER BY notDischarged desc
)

)

'''

data_sql = pd.read_sql_query(sql,con=engine)
data_sql

,States,notDischarged
0,Punjab,0.024631
1,Punjab,0.024631
2,Nagaland,0.021731
3,Nagaland,0.021731
4,Maharashtra,0.018257
...,...,...
67,Arunachal Pradesh,0.004425
68,Mizoram,0.003038
69,Mizoram,0.003038
70,Dadra and Nagar Haveli and Daman and Diu,0.000345


In [63]:
# Query 7 - comparing regionally, the active rates between south india and north 
sql = '''

SELECT 'South India Active Rate: ' || States, avg(CAST(Active AS FLOAT)/CAST(Population AS FLOAT)) as RateOfActiveCases 
FROM IndiaCovidData
WHERE States in ('Tamil Nadu','Puducherry','Telengana','Andhra Pradesh','Karnataka','Kerala')
union all 
SELECT 'North India Active Rate: ' || States, avg(CAST(Active AS FLOAT)/CAST(Population AS FLOAT)) as RateOfActiveCases 
FROM IndiaCovidData
WHERE States not in ('Tamil Nadu','Puducherry','Telengana','Andhra Pradesh','Karnataka','Kerala')
'''

data_sql = pd.read_sql_query(sql,con=engine)
data_sql

,'South India Active Rate: ' || States,RateOfActiveCases
0,South India Active Rate: Andhra Pradesh,0.000021
1,North India Active Rate: Andaman and Nicobar,0.000018


In [64]:
# Query 8 - comparing regionally, the active ratio between south india and north (South : North)
sql = '''

SELECT RateOfActiveCases
FROM(
SELECT 'South India Active Rate: ' || States, avg(CAST(Active AS FLOAT)/CAST(Population AS FLOAT)) as RateOfActiveCases 
FROM IndiaCovidData
WHERE States in ('Tamil Nadu','Puducherry','Telengana','Andhra Pradesh','Karnataka','Kerala')
union all 
SELECT 'North India Active Rate: ' || States, avg(CAST(Active AS FLOAT)/CAST(Population AS FLOAT)) as RateOfActiveCases 
FROM IndiaCovidData
WHERE States not in ('Tamil Nadu','Puducherry','Telengana','Andhra Pradesh','Karnataka','Kerala')
)

'''
data_sql = pd.read_sql_query(sql,con=engine)
data_sql.iloc[0]/data_sql.iloc[1]

RateOfActiveCases    1.14148
dtype: float64

In [65]:
# Query 9 - comparing regionally, the death ratio between south india and north (South : North)
sql = '''

SELECT notDischarged
FROM(
SELECT 'South India Active Rate: ' || States, 1 - avg(CAST(Discharged AS FLOAT)/CAST(TotalCases AS FLOAT)) as notDischarged 
FROM IndiaCovidData
WHERE States in ('Tamil Nadu','Puducherry','Telengana','Andhra Pradesh','Karnataka','Kerala')
union all 
SELECT 'North India Active Rate: ' || States, 1 - avg(CAST(Discharged AS FLOAT)/CAST(TotalCases AS FLOAT)) as notDischarged 
FROM IndiaCovidData
WHERE States not in ('Tamil Nadu','Puducherry','Telengana','Andhra Pradesh','Karnataka','Kerala')
)
'''
data_sql = pd.read_sql_query(sql,con=engine)
data_sql.iloc[0]/data_sql.iloc[1]

notDischarged    0.785074
dtype: float64